In [10]:
import os
import pandas as pd
import numpy as np

class market():
    def word_cut(self,documents,stopwords = stopwords):
        import jieba
        texts = []
        for line in documents:
            words = ' '.join(jieba.cut(line)).split(' ') # 用空格去连接，连接后马上又拆分
            text = []
            for word in words:
                if (word not in stopwords) & (word != '')& (word != '\u3000')& (word != '\n')&(word != '\u200b'):
                    text.append(word)
            texts.append(text)
        self.docLength = len(documents)
        return(texts)
    def get_docLength(self):
        return(self.docLength)
    def frequency(self,texts,freq):
        from collections import defaultdict
        frequency = defaultdict(int) # value为int
        for text in texts:
            for word in text:
                frequency[word] += 1
        texts = [[word for word in text if frequency[word] > freq] for text in texts]
        return(texts)
    def regroup(self,texts):
        new_texts = []
        for i,sentence in enumerate(texts):
            new_texts.append(" ".join(sentence))
        return(new_texts)
    def add_stopwords(self,path):
        stopwords = set()
        with open(path,'r',encoding = 'cp936') as file:
            for line in file:
                stopwords.add(line.strip())
        return(stopwords)
    def dictionary(self,docs):
        token_index ={}
        for sample in docs:
            for word in sample:
                if word not in token_index:
                    token_index[word] = len(token_index) + 1
        return(token_index)
    def count(self,docs):
        token_length ={}
        for sample in docs:
            for word in sample:
                if word not in token_length:
                    token_length[word] = 1
                else:
                    token_length[word] += 1
        return(token_length)
    def recoding(self,docs,token_index):
        for i,sample in enumerate(docs):
            for j,word in enumerate(sample):
                if word not in token_index:
                    sample[j] = -1
                else:
                    sample[j] = token_index[word]
            docs[i] = sample
        return(docs)
    def delete(self,docs):
        for index in range(len(docs)):
            for i in range(len(docs[index])-1,-1,-1):
                if docs[index][i] == -1:
                    docs[index].pop(i)
        return docs
    def random_pick(self,df,n):
        import random
        import numpy as np
        rand = np.arange(0,(len(df)-1),1)
        random.shuffle(rand)
        rand = list(rand[:n])
        df = df.loc[rand,]
        return(df)    
    def navie_knn(self,dataSet, query, k):  
        # 计算出某一样本与所有样本的距离，选择最大(应该修改为最小？)的k个样本作为用于knn
        numSamples = dataSet.shape[0] # return row(sample) number of dataset

        ## step 1: calculate Euclidean distance  
        diff = np.tile(query, (numSamples, 1)) - dataSet #tile: 把query这个向量纵向复制，使得结果与dataset具有同样的行数
        squaredDiff = diff ** 2  
        squaredDist = np.sum(squaredDiff, axis = 1) # sum is performed by row  

        ## step 2: sort the distance  
        sortedDistIndices = np.argsort(squaredDist)   # numpy.argsort 返回的是数组值从小到大的索引值（注意是索引值，不是绝对值）
        if k > len(sortedDistIndices):  
            k = len(sortedDistIndices)  

        return sortedDistIndices[0:k]
    # build a big graph (normalized weight matrix)  
    def buildGraph(self,MatX, kernel_type, rbf_sigma = None, knn_num_neighbors = None):  
        num_samples = MatX.shape[0]  # return row(sample) number of MatX
        affinity_matrix = np.zeros((num_samples, num_samples), np.float32)  
        if kernel_type == 'rbf':  
            if rbf_sigma == None:  
                raise ValueError('You should input a sigma of rbf kernel!')  
            for i in range(num_samples):  
                row_sum = 0.0  
                for j in range(num_samples):  
                    diff = MatX[i, :] - MatX[j, :]  
                    affinity_matrix[i][j] = np.exp(sum(diff**2) / (-2.0 * rbf_sigma**2))  
                    row_sum += affinity_matrix[i][j]  
                affinity_matrix[i][:] /= row_sum  
        elif kernel_type == 'knn':  
            if knn_num_neighbors == None:  
                raise ValueError('You should input a k of knn kernel!')  
            for i in range(num_samples):  
                k_neighbors = self.navie_knn(MatX, MatX[i, :], knn_num_neighbors)  
                affinity_matrix[i][k_neighbors] = 1.0 / knn_num_neighbors  # 将节点i与附近的k个节点连接起来，每个边的权重是1/knn_num_neighbors
        else:  
            raise NameError('Not support kernel type! You can use knn or rbf!')  

        return affinity_matrix  
    # label propagation  
    def labelPropagation(self,Mat_Label, Mat_Unlabel, labels, kernel_type = 'rbf', rbf_sigma = 0.20, \
                        knn_num_neighbors = 10, max_iter = 500, tol = 1e-3):  
        # initialize  
        num_label_samples = Mat_Label.shape[0]  #已经标记的sample number
        num_unlabel_samples = Mat_Unlabel.shape[0]  #未标记的sample number
        num_samples = num_label_samples + num_unlabel_samples
        labels_list = np.unique(labels)  #有哪些label
        num_classes = len(labels_list)  #label的种类数

        MatX = np.vstack((Mat_Label, Mat_Unlabel))
        clamp_data_label = np.zeros((num_label_samples, num_classes), np.float32)  
        for i in range(num_label_samples):  
            clamp_data_label[i][labels[i]] = 1.0   #标记出每一个labelled sample的具体label是什么

        label_function = np.zeros((num_samples, num_classes), np.float32)  
        label_function[0 : num_label_samples] = clamp_data_label  
        label_function[num_label_samples : num_samples] = -1  

        # graph construction  
        affinity_matrix = self.buildGraph(MatX, kernel_type, rbf_sigma, knn_num_neighbors)  

        # start to propagation  
        iter = 0; pre_label_function = np.zeros((num_samples, num_classes), np.float32)  
        changed = np.abs(pre_label_function - label_function).sum()  
        while iter < max_iter and changed > tol:  
            if iter % 1 == 0:  
                print ("---> Iteration %d/%d, changed: %f" % (iter, max_iter, changed))
            pre_label_function = label_function  
            iter += 1  

            # propagation  
            label_function = np.dot(affinity_matrix, label_function)  

            # clamp  
            label_function[0 : num_label_samples] = clamp_data_label  

            # check converge  
            changed = np.abs(pre_label_function - label_function).sum()  

        # get terminate label of unlabeled data  
        unlabel_data_labels = np.zeros(num_unlabel_samples)  
        for i in range(num_unlabel_samples):  
            unlabel_data_labels[i] = np.argmax(label_function[i+num_label_samples]) #取出参数中元素最大值所对应的索引 

        return unlabel_data_labels 

In [11]:
process = market()

os.chdir("D:/Users/PYTHON/Precision-Marketing")

WORRY = pd.read_csv("已标记的担忧咋办标签.csv")
WORRY = WORRY[["博文","是否担忧（1=担忧，0=不担忧）"]] 
PATIENT = pd.read_csv("已标记的会不会得病标签.csv")
PATIENT = PATIENT[["博文","是否担忧（1=担忧，0=不担忧）"]]

print(WORRY.head(5))
print(PATIENT.head(5))

                                                  博文  是否担忧（1=担忧，0=不担忧）
0  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...                 0
1  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021啊好...                 0
2  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...                 0
3  问：如何让猫黏上你？答：穿一条毛茸茸的裤子。————但是我现在有点担心你现在睡这么多晚上我睡...                 0
4  今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...                 0
                                                  博文  是否担忧（1=担忧，0=不担忧）
0  给好多人说了新年快乐说了新年祝福还没有给自己说过。新年快乐希望在2020年你可以学有所成重要...               0.0
1  2019开始倒数了哦原来小时候觉得很遥远的2020年就是明天了要说2019年我收获了什么呢？...               1.0
2  微信，QQ都是认识的人，没有办法发泄内心的不满，渐渐养成了不发朋友圈的习惯，但是自己变得越来...               1.0
3  现在已经对酒上瘾很严重每天这个点必须喝半杯而且一天喝水喝特别少大概就一瓶不到..我会不会得病...               1.0
4  这段时间每天睡得晚，起的晚，可恨没有好好吃早餐。会不会得病啊中午上一节男老师的尊巴舞，如果体...               1.0


In [12]:
df = pd.concat([WORRY,PATIENT]) # concat之后row index会重复
# df = pd.concat([PATIENT])
df.reset_index(drop = True,inplace = True)
print(len(df))
df = df.dropna()
print(len(df))
df.head(5)

1049497
1636


,博文,是否担忧（1=担忧，0=不担忧）
0,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0.0
1,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021啊好...,0.0
2,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0.0
3,问：如何让猫黏上你？答：穿一条毛茸茸的裤子。————但是我现在有点担心你现在睡这么多晚上我睡...,0.0
4,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0.0


In [13]:
df["是否担忧（1=担忧，0=不担忧）"].value_counts()

0.0    1370
1.0     266
Name: 是否担忧（1=担忧，0=不担忧）, dtype: int64

In [18]:
df_worry = df[df["是否担忧（1=担忧，0=不担忧）"] == 1]
df_non_worry = df[df["是否担忧（1=担忧，0=不担忧）"] == 0]
df_non_worry = process.random_pick(df_non_worry,min(len(df_worry),len(df_non_worry)))
print(len(df_worry))
print(len(df_non_worry))

266
266


In [19]:
df_worry

,博文,是否担忧（1=担忧，0=不担忧）
65,焦虑症我咋办啊!!我又开始怀疑自己心脏病了…每时每刻都在担心自己是不是会突然倒下去这几天都...,1.0
122,累…要做好工作还要当好妈妈，有多久没有凌晨两点前睡过觉了，刚收拾完宝贝明天去海洋馆要拿的东西...,1.0
207,我是真的很担心我的胳膊不好使了咋办2武汉·亿童大厦​,1.0
209,臭宝前天晚上去医院看了，大夫说是支原体感染，在网上查说支原体感染很麻烦，很容易复发，我又担心...,1.0
253,36个年头，第一次这么怕死，虽然问过医生，还是有点担心，怕真有什么，俩宝咋办，四老咋办，唉，...,1.0
...,...,...
1607,刚刚收到公司行政部的短信调查，问我感冒咳嗽好了没。啊啊啊，好害怕回去上班！路上好危险，要搭地...,1.0
1609,你们在家里呆有没有出现很闷 就是心慌然后呼吸急促 怎么办我好害怕😨 ​,1.0
1612,妈妈不在家的时候我总是很焦虑，我害怕她出事，虽然她身体健康，我想和她每天每个时间都在一起，妈...,1.0
1631,望着躺在重症监护室里的父亲，心里五味杂陈，我笑着告诉他一切都好，却害怕他看见我眼角的泪有些话...,1.0


In [20]:
df_non_worry

,博文,是否担忧（1=担忧，0=不担忧）
506,【胡闹调侃网红办活动随礼多收不回来】二嫂和胡闹直播提及网红办活动，胡闹喊话以后你们办活动再拉...,0.0
980,老师：加课我：去补习前的我：7点半跨年杨幂出来你录一下我妹：补习中我：害杨幂出来了咋办我朋友...,0.0
776,慌的一笔，我果然有毒，我不能看比赛，我现在有点担心总决赛我在现场咋办了....2苏州​,0.0
957,【66】今天终于把史纲看完了最后一部分内容就比较少了加快速度英语才做完12年唯一欣慰的就是这...,0.0
657,【居住储藏间，遭装电难题！这事该咋办？】家住福清的吴女士去年在福清买了个储藏间，可是房子购买...,0.0
...,...,...
139,蔡徐坤#蔡徐坤江苏卫视跨年#起床第一件事，就是应援灯，终于拥有啦！好怕和大家抢东西，前两次...,0.0
1044,我家这个网卡的我担心我过两天上网课咋办平时刷手机还能用流量上网课我流量真的顶不住啊​,0.0
132,有时候还是会不由自主的担心，大了发现他应发也不好，喊好久不理，还是看不懂表情，也不会点头摇头...,0.0
655,真的是第一次和狗子生活而且还是在陌生的地方今晚傻狗莫名其妙向着对面来的一只体型比它小一点的狗...,0.0


In [21]:
df_worry = df_worry.dropna()
df_non_worry = df_non_worry.dropna()
print(len(df_worry))
print(len(df_non_worry))

df_use = pd.concat([df_worry,df_non_worry])
df_use.reset_index(drop = True,inplace = True)
df_use = df_use.reindex(np.random.permutation(df_use.index))
df_use.head()


266
238


,博文,是否担忧（1=担忧，0=不担忧）
155,越来开心点越难做到，最近总是肚子胀手脚麻，会不会得什么得病，我要是病了谁能照顾孩子啊！真心谁...,1.0
304,荷兰要改名，我是没啥意见。就是担心，荷兰豆、荷兰弟、荷兰猪，他们该咋办呢？​,0.0
210,我太喜欢吃炸串烧烤可乐这些不健康食物了，可是又忍不住，每天担心自己会不会得病​,1.0
382,天哪，我该咋办抑郁症不是我能承受的来的呀，我要怎么面对她我实在是担心我的这点儿也被拉下水啊坏...,0.0
272,喜欢不定时地清理相册把照片都存QQ空间喜欢手机里边空荡荡的不过很担心哪天空间存不下了咋办​,0.0


In [23]:
stopwords = process.add_stopwords("D:/Users/PYTHON/Precision-Marketing/stopwords.txt")
x_train = process.word_cut(df_use["博文"],stopwords)
x_train = process.frequency(x_train,5)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.627 seconds.
Prefix dict has been built successfully.


In [24]:
x_train[:5]

[['越', '开心', '肚子', '得病', '病', '照顾', '孩子'],
 ['没', '担心', '咋办'],
 ['太', '喜欢', '健康', '担心', '得病'],
 ['天', '咋办', '抑郁症', '我要', '实在', '担心', '坏', '情绪', '太'],
 ['喜欢', '喜欢', '手机', '担心', '天', '咋办']]

In [25]:
token_index = process.dictionary(x_train)
token_index

{'越': 1,
 '开心': 2,
 '肚子': 3,
 '得病': 4,
 '病': 5,
 '照顾': 6,
 '孩子': 7,
 '没': 8,
 '担心': 9,
 '咋办': 10,
 '太': 11,
 '喜欢': 12,
 '健康': 13,
 '天': 14,
 '抑郁症': 15,
 '我要': 16,
 '实在': 17,
 '坏': 18,
 '情绪': 19,
 '手机': 20,
 '刷': 21,
 '脸': 22,
 '有人': 23,
 '早上': 24,
 '不想': 25,
 '学习': 26,
 '昨天': 27,
 '视频': 28,
 '里': 29,
 '说': 30,
 '快': 31,
 '抢': 32,
 '哭': 33,
 '感觉': 34,
 '差': 35,
 '好多': 36,
 '抑郁': 37,
 '收起': 38,
 '全文': 39,
 'd': 40,
 '妈妈': 41,
 '再也': 42,
 '不用': 43,
 '月': 44,
 '宝宝': 45,
 '时': 46,
 '身体': 47,
 '影响': 48,
 '爱': 49,
 '吃': 50,
 '想到': 51,
 '上班': 52,
 '娃': 53,
 '真的': 54,
 '一年': 55,
 '生活': 56,
 '累': 57,
 '身边': 58,
 '幸福': 59,
 '想': 60,
 '好好': 61,
 '特别': 62,
 '害怕': 63,
 '爸妈': 64,
 '生病': 65,
 '舒服': 66,
 '我会': 67,
 '希望': 68,
 '有时候': 69,
 '话': 70,
 '朋友': 71,
 '老': 72,
 '出': 73,
 '带': 74,
 '狗子': 75,
 '回来': 76,
 '想想': 77,
 '生': 78,
 '儿子': 79,
 '忙': 80,
 '死': 81,
 'L': 82,
 '微博': 83,
 '一点': 84,
 '饭': 85,
 '两个': 86,
 '明天': 87,
 '吃饭': 88,
 '两天': 89,
 '掉': 90,
 '天气': 91,
 '跑': 92,
 '体验': 93,
 '秒': 94,
 '拍': 9

In [26]:
len(token_index)

376

In [27]:
token_length = process.count(x_train)
token_length = {key:token_length[key] for key in sorted(token_length,key = lambda x: token_length[x],reverse = True)[:round((2/5)*len(token_index))]}
token_length

{'担心': 299,
 '得病': 271,
 '咋办': 226,
 '说': 142,
 '想': 134,
 '真的': 110,
 '收起': 101,
 '全文': 101,
 'd': 101,
 '没': 86,
 '吃': 76,
 '感觉': 71,
 '我会': 67,
 '害怕': 58,
 '太': 47,
 '做': 47,
 '孩子': 42,
 '视频': 41,
 '妈妈': 40,
 '生活': 37,
 '怀疑': 37,
 '死': 36,
 '完': 36,
 '月': 35,
 '总': 35,
 '怕': 35,
 '睡': 32,
 '开心': 30,
 '哭': 30,
 '微博': 30,
 '一点': 30,
 '不好': 29,
 '睡觉': 29,
 'L': 28,
 '年': 28,
 '宝宝': 27,
 '晚上': 27,
 '发现': 27,
 '更': 27,
 '天天': 27,
 '喜欢': 26,
 '特别': 26,
 '医生': 26,
 '里': 25,
 '活动': 25,
 '不想': 24,
 '事': 24,
 '焦虑': 24,
 '胡闹': 24,
 '好像': 23,
 '�': 23,
 '手机': 22,
 '身体': 22,
 '真': 22,
 '东西': 22,
 '事情': 22,
 '爱': 21,
 '好好': 21,
 '回来': 21,
 '明天': 21,
 '点': 21,
 '钱': 21,
 '工作': 21,
 '有时候': 20,
 '买': 20,
 '时间': 20,
 '想着': 20,
 '走': 20,
 '前': 20,
 '情绪': 19,
 '希望': 19,
 '老': 19,
 '难受': 19,
 '\ue627': 19,
 '心情': 19,
 '出': 18,
 '越来越': 18,
 '两天': 17,
 '咳嗽': 17,
 '家里': 17,
 '回家': 17,
 '上海': 17,
 '每次': 17,
 '病': 16,
 '天': 16,
 '上班': 16,
 '朋友': 16,
 '带': 16,
 '睡不着': 16,
 'O': 16,
 '中': 16,
 '难过': 16,
 '问': 

In [28]:
# 给字符编码。如果x_train中的词语不在传入的实参token_index中，那么就编码为-1
x_train = process.recoding(x_train,token_length)
x_train[:5]

[[13, 30, -1, 271, 16, 10, 42],
 [86, 299, 226],
 [47, 26, 14, 299, 271],
 [16, 226, -1, 10, -1, 299, -1, 19, 47],
 [26, 26, 22, 299, 16, 226]]

In [29]:
x_train = process.delete(x_train)
x_train[:2]

[[13, 30, 271, 16, 10, 42], [86, 299, 226]]

In [30]:
import keras
import tensorflow
from keras import preprocessing

max_len = 50
x_train = preprocessing.sequence.pad_sequences(x_train,maxlen = max_len)

x_train.shape

Using TensorFlow backend.


(504, 50)

In [31]:
y_train = df_use[["是否担忧（1=担忧，0=不担忧）"]]
# y_train["是否担忧（1=担忧，0=不担忧）"] = y_train["是否担忧（1=担忧，0=不担忧）"].astype("category")
y_train.head(5)

,是否担忧（1=担忧，0=不担忧）
155,1.0
304,0.0
210,1.0
382,0.0
272,0.0


In [32]:
y_in = len(y_train)
y_train = np.array(y_train)
y_train = y_train.reshape(y_in)
y_train[:5]

array([1., 0., 1., 0., 0.])

In [33]:
print(x_train.shape)
print(y_train.shape)

(504, 50)
(504,)


In [34]:
def get_values(token_index):
    values = []
    for key in token_index:
        values.append(token_index[key])
    return(values)
values = get_values(token_index)
values[:5]
print("最大的序号是：",max(values))

最大的序号是： 376


In [37]:
from keras.models import Sequential
from keras.layers import Flatten,Dense,Embedding,LSTM,Bidirectional,Dropout

max_features = 1000
max_len = 50

model = Sequential()
model.add(Embedding(max_features,8,input_length = max_len,mask_zero = True)) # 遇到0，就不会反向传播更新权重
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(20),merge_mode = 'concat'))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 8)             8000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 8)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40)                4640      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 41        
Total params: 12,681
Trainable params: 12,681
Non-trainable params: 0
_________________________________________________________________


In [42]:
history = model.fit(x_train,
                    y_train,
                    epochs = 5,
                    batch_size = 128,
                    validation_split=0)

Epoch 1/5
504/504 [==============================] - 0s 359us/step - loss: 0.4960 - acc: 0.8889
Epoch 2/5
504/504 [==============================] - 0s 324us/step - loss: 0.4815 - acc: 0.8810
Epoch 3/5
504/504 [==============================] - 0s 291us/step - loss: 0.4601 - acc: 0.8988
Epoch 4/5
504/504 [==============================] - 0s 291us/step - loss: 0.4419 - acc: 0.8948
Epoch 5/5
504/504 [==============================] - 0s 281us/step - loss: 0.4485 - acc: 0.8889


In [71]:
# model.save("Bi-LSTM.h5")

In [43]:
new_data = pd.read_csv("未标记的担忧咋办标签.csv")
new_data.head(2)

,用户名,博文,预测是否担忧（1=担忧，0=不担忧）,转发数,评论数,点赞数,发文时间,来自,页面网址,博文链接
0,玥玥的碎花小裙儿,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021年啊...,0,,,,\n 2019年12月31日 23:52\n ...,realme Q 四摄迅猛龙,https://s.weibo.com/weibo/%25E6%258B%2585%25E5...,https://weibo.com/6275719793/Innh3oI0n?refer_f...
1,海里星星16687,今晚就要跨年了咋办呢？先迈左脚还是右脚啊？跨不过去会不会摔一跤？腿太长会不会跨到2021啊好...,0,,,,\n 2019年12月31日 23:35\n ...,OPPO超视野全面屏R15,https://s.weibo.com/weibo/%25E6%258B%2585%25E5...,https://weibo.com/6607786597/InnadlLNd?refer_f...


In [45]:
new_data = new_data[["博文"]]
new_data["博文"] = process.word_cut(new_data["博文"],stopwords = stopwords)
new_data.head(2)

,博文
0,"[今晚, 跨年, 咋办, 先迈, 左脚, 右脚, 跨, 摔, 一跤, 腿, 太长会, 跨到,..."
1,"[今晚, 跨年, 咋办, 先迈, 左脚, 右脚, 跨, 摔, 一跤, 腿, 太长会, 跨到,..."


In [46]:
new_data["博文"] = process.frequency(new_data["博文"],5)
new_data.head(2)

,博文
0,"[今晚, 跨年, 咋办, 先迈, 左脚, 右脚, 跨, 摔, 一跤, 腿, 太长会, 跨到,..."
1,"[今晚, 跨年, 咋办, 先迈, 左脚, 右脚, 跨, 摔, 一跤, 腿, 太长会, 跨到,..."


In [47]:
new_data["博文"] = process.recoding(new_data["博文"],token_length)
new_data.head(2)

,博文
0,"[-1, -1, 226, -1, -1, -1, 11, -1, -1, 11, -1, ..."
1,"[-1, -1, 226, -1, -1, -1, 11, -1, -1, 11, -1, ..."


In [48]:
new_data["博文"] = process.delete(new_data["博文"])

In [52]:
x_test = preprocessing.sequence.pad_sequences(new_data["博文"],maxlen = max_len)

In [65]:
prediction = model.predict(x_test)
prediction = prediction.reshape(len(prediction))
prediction

array([0.28638244, 0.3561222 , 0.28638244, ..., 0.22269207, 0.4978666 ,
       0.36481923], dtype=float32)

In [67]:
list(prediction)

[0.28638244,
 0.3561222,
 0.28638244,
 0.3070818,
 0.28638244,
 0.28638244,
 0.036987633,
 0.28638244,
 0.041386187,
 0.28638244,
 0.20439968,
 0.28638244,
 0.36905697,
 0.36905697,
 0.17804673,
 0.37612838,
 0.20604452,
 0.372408,
 0.28638244,
 0.17126405,
 0.2568181,
 0.36910245,
 0.28638244,
 0.12604931,
 0.4303737,
 0.38827926,
 0.26504618,
 0.3822643,
 0.32645917,
 0.030644298,
 0.11945096,
 0.28638244,
 0.28638244,
 0.36720484,
 0.12581888,
 0.28638244,
 0.47118962,
 0.37029147,
 0.158634,
 0.47752842,
 0.10418078,
 0.3545034,
 0.3565116,
 0.26350534,
 0.20915121,
 0.29093534,
 0.3411907,
 0.36905697,
 0.20915121,
 0.2146118,
 0.49749994,
 0.20915121,
 0.20915121,
 0.19770297,
 0.3375891,
 0.2146118,
 0.062829256,
 0.20915121,
 0.20551956,
 0.20915121,
 0.36905697,
 0.583745,
 0.35931066,
 0.36905697,
 0.36185318,
 0.04843709,
 0.36056793,
 0.23598874,
 0.33424455,
 0.22741976,
 0.16428724,
 0.34336352,
 0.2318049,
 0.44050083,
 0.3524977,
 0.3923841,
 0.3949593,
 0.3518719,
 0.2

In [69]:
Result = pd.DataFrame({"预测结果":list(prediction)})
# Result.to_excel("Bi-LSTM预测结果.xlsx",header = True)